In [1]:
'''
Author: Aditya Jain
Date  : 11th June, 2021
About : This script predicts the class given an input image
'''
import sys, os
sys.path.append('/home/mila/a/aditya.jain/mothAI/deeplearning')

from PIL import Image
import matplotlib.pyplot as plt
from torchvision import transforms
import torch
from models.resnet50 import Resnet50
import json
import numpy as np

image_path   = 'test_image.jpg'
image_resize = 224
config_file  = '/home/mila/a/aditya.jain/mothAI/deeplearning/config/01-config.json' 
PATH         = '/home/mila/a/aditya.jain/logs/v01_mothmodel_2021-06-08-04-53.pt'
label_file   = '/home/mila/a/aditya.jain/mothAI/deeplearning/data/numeric_labels.json'

/home/mila/a/aditya.jain/.conda/envs/milamoth_ai/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'models'

In [10]:
# model loading
f           = open(config_file)
config_data = json.load(f)

device      = "cuda" if torch.cuda.is_available() else "cpu"
model       = Resnet50(config_data).to(device)

checkpoint  = torch.load(PATH, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [12]:
# image loading and processing
image     = Image.open(image_path)

transform = transforms.Compose([
                        transforms.Resize((image_resize, image_resize)),              # resize the image to 224x224 
                        transforms.ToTensor()])
image     = transform(image)
image     = torch.unsqueeze(image, 0).to(device)

In [21]:
# prediction
prediction      = model(image)
_, predict_indx = torch.topk(prediction, 1)

In [38]:
f            = open(label_file)
label_info   = json.load(f)
species_list = label_info['species_list']

print('The predicted species is : ', species_list[predict_indx])

The predicted species is :  Sphinx luscitiosa


In [48]:
# finding top 3 predictions
_, predict_indx = torch.topk(prediction, 3)
predict_indx    = predict_indx[0]

for index in predict_indx:
    print(species_list[index])

Sphinx luscitiosa
Hemileuca lucina
Proserpinus flavofasciata
